In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures

In [2]:
X_train = pd.read_csv('ds6-league-of-legends/participants_train.csv')
X_test = pd.read_csv('ds6-league-of-legends/participants_test.csv')
y_train = pd.read_csv('ds6-league-of-legends/train_winners.csv')
submission = pd.read_csv('ds6-league-of-legends/submission.csv')
champion_mastery = pd.read_csv('ds6-league-of-legends/champion_mastery.csv')
Position = pd.read_csv('ds6-league-of-legends/teamPositions.csv')
X_train_timeline = pd.read_csv('ds6-league-of-legends/train_timeline_values.csv')
Y_test_timeline = pd.read_csv('ds6-league-of-legends/test_timeline_values.csv')
champions = pd.read_json('ds6-league-of-legends/champion.json')

In [3]:
#creating a copy of the X data to avoid corruption in the dataset
X_train_original = X_train
y_train_original = y_train
X_test_original = X_test

# this is to extract the columns that is needed for training
y_train = y_train['winner']

In [4]:
variables = ['summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg',
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl'
             ]

# found out later these are the important ones:
# variables = ['summonerLevel', 'championLevel', 'final_gold', 'final_xp', 'final_armor', 'final_health', 'final_atkdmg']


X_train = pd.merge(X_train_original, X_train_timeline, how='inner', on=['matchId', 'participantId'])
X_train = pd.merge(X_train, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_train = X_train.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_train = (
    X_train
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)


#X_train = X_train.groupby('matchId')[variables].sum().reset_index(drop = True)


X_train

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
0,0,100,860,30.0,14146,17159,316,409,6268,11867,21635,375,940,34
1,0,200,532,30.0,17539,18773,289,579,6040,13396,21385,943,374,35
2,1,100,570,33.0,17662,17770,255,511,5636,11445,16807,1272,959,34
3,1,200,567,25.0,14816,18028,316,463,6414,8987,22478,959,1272,34
4,2,100,619,25.0,16374,16837,292,556,5913,14196,23323,195,483,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,7997,200,464,29.0,16571,15721,352,534,5940,13150,24331,1715,1248,33
15996,7998,100,689,26.0,15543,15853,277,411,6585,10665,21440,211,346,32
15997,7998,200,1165,32.0,18311,18654,278,553,6067,15595,17865,347,211,36
15998,7999,100,991,26.0,17058,19319,304,517,5805,10454,17554,228,591,35


In [5]:
team1 = X_train[X_train['teamId'] == 100]
team1

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
0,0,100,860,30.0,14146,17159,316,409,6268,11867,21635,375,940,34
2,1,100,570,33.0,17662,17770,255,511,5636,11445,16807,1272,959,34
4,2,100,619,25.0,16374,16837,292,556,5913,14196,23323,195,483,32
6,3,100,515,28.0,19247,17562,348,531,6240,16263,23900,1020,890,34
8,4,100,747,28.0,16776,18264,304,453,6292,11343,21601,163,800,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15990,7995,100,772,30.0,19009,18803,290,531,6395,16252,20485,94,722,36
15992,7996,100,533,28.0,16413,17298,426,426,6367,14939,19961,309,1015,33
15994,7997,100,567,29.0,17745,17090,302,556,5918,16493,18230,1249,1715,34
15996,7998,100,689,26.0,15543,15853,277,411,6585,10665,21440,211,346,32


In [6]:
team2 = X_train[X_train['teamId'] == 200]
team2

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
1,0,200,532,30.0,17539,18773,289,579,6040,13396,21385,943,374,35
3,1,200,567,25.0,14816,18028,316,463,6414,8987,22478,959,1272,34
5,2,200,658,24.0,17347,17467,317,527,6500,15374,24127,483,195,34
7,3,200,599,31.0,15145,16196,301,530,5760,13223,23420,890,1020,32
9,4,200,352,21.0,15385,17194,300,583,5817,12364,20043,800,163,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15991,7995,200,898,30.0,14873,16977,304,509,5916,12284,23980,723,94,34
15993,7996,200,873,30.0,16140,17927,315,573,6110,12364,23494,1015,309,34
15995,7997,200,464,29.0,16571,15721,352,534,5940,13150,24331,1715,1248,33
15997,7998,200,1165,32.0,18311,18654,278,553,6067,15595,17865,347,211,36


In [7]:
X_teams = pd.merge(team1, team2, how='outer', on=['matchId'])

X_teams

,matchId,teamId_x,summonerLevel_x,championLevel_x,final_gold_x,final_xp_x,final_armor_x,final_atkdmg_x,final_health_x,final_dmgdone_x,...,final_gold_y,final_xp_y,final_armor_y,final_atkdmg_y,final_health_y,final_dmgdone_y,final_dmgtaken_y,final_truedmgdone_y,final_truedmgtaken_y,final_lvl_y
0,0,100,860,30.0,14146,17159,316,409,6268,11867,...,17539,18773,289,579,6040,13396,21385,943,374,35
1,1,100,570,33.0,17662,17770,255,511,5636,11445,...,14816,18028,316,463,6414,8987,22478,959,1272,34
2,2,100,619,25.0,16374,16837,292,556,5913,14196,...,17347,17467,317,527,6500,15374,24127,483,195,34
3,3,100,515,28.0,19247,17562,348,531,6240,16263,...,15145,16196,301,530,5760,13223,23420,890,1020,32
4,4,100,747,28.0,16776,18264,304,453,6292,11343,...,15385,17194,300,583,5817,12364,20043,800,163,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,100,772,30.0,19009,18803,290,531,6395,16252,...,14873,16977,304,509,5916,12284,23980,723,94,34
7996,7996,100,533,28.0,16413,17298,426,426,6367,14939,...,16140,17927,315,573,6110,12364,23494,1015,309,34
7997,7997,100,567,29.0,17745,17090,302,556,5918,16493,...,16571,15721,352,534,5940,13150,24331,1715,1248,33
7998,7998,100,689,26.0,15543,15853,277,411,6585,10665,...,18311,18654,278,553,6067,15595,17865,347,211,36


In [8]:
variables = ['summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg',
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl'
             ]

# found out later these are the important ones:
# variables = ['summonerLevel', 'championLevel', 'final_gold', 'final_xp', 'final_armor', 'final_health', 'final_atkdmg']


X_test = pd.merge(X_test_original, Y_test_timeline, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_test = X_test.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_test = (
    X_test
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

#X_test = X_test.groupby('matchId')[variables].sum().reset_index(drop = True)



In [9]:
X_test1 =  X_test[X_test['teamId'] == 100]
X_test1

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
0,8000,100,788,31.0,17313,18099,288,510,6304,11079,18916,604,520,35
2,8001,100,685,31.0,17492,16756,264,499,5584,14834,19547,176,160,33
4,8002,100,612,29.0,16206,16640,303,494,5869,15843,24991,338,376,33
6,8003,100,707,30.0,17784,16759,293,565,6250,17100,23946,1498,604,33
8,8004,100,611,27.0,16341,18377,267,454,5802,13477,17305,687,406,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3990,9995,100,905,35.0,15440,17688,279,414,6072,13331,19797,1835,743,34
3992,9996,100,560,28.0,19123,18273,291,541,6542,13467,23297,1566,76,35
3994,9997,100,510,25.0,14719,17211,282,512,5914,7891,23464,394,522,35
3996,9998,100,556,30.0,18353,18298,344,625,6043,13838,20754,679,70,35


In [10]:
X_test2 =  X_test[X_test['teamId'] == 200]
X_test2

,matchId,teamId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,final_dmgdone,final_dmgtaken,final_truedmgdone,final_truedmgtaken,final_lvl
1,8000,200,685,28.0,16933,17117,272,508,5731,12235,21465,521,604,33
3,8001,200,539,26.0,17533,16527,285,424,6141,14326,21925,160,176,33
5,8002,200,653,25.0,17480,16518,268,509,6248,17065,23649,376,338,33
7,8003,200,1339,23.0,18126,17173,402,521,6314,16230,25839,604,1497,34
9,8004,200,660,27.0,16993,17113,324,523,6612,11488,20156,406,687,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,9995,200,613,25.0,15060,17845,285,449,6519,11433,21529,743,1835,35
3993,9996,200,697,30.0,17433,18545,328,566,6238,14474,23791,76,1566,36
3995,9997,200,574,26.0,17129,18888,269,507,5629,14181,18033,523,394,35
3997,9998,200,655,30.0,15134,16823,293,495,6042,12048,23098,70,679,34


In [11]:
y_teams_test = pd.merge(X_test1, X_test2, how='outer', on=['matchId'])

y_teams_test

,matchId,teamId_x,summonerLevel_x,championLevel_x,final_gold_x,final_xp_x,final_armor_x,final_atkdmg_x,final_health_x,final_dmgdone_x,...,final_gold_y,final_xp_y,final_armor_y,final_atkdmg_y,final_health_y,final_dmgdone_y,final_dmgtaken_y,final_truedmgdone_y,final_truedmgtaken_y,final_lvl_y
0,8000,100,788,31.0,17313,18099,288,510,6304,11079,...,16933,17117,272,508,5731,12235,21465,521,604,33
1,8001,100,685,31.0,17492,16756,264,499,5584,14834,...,17533,16527,285,424,6141,14326,21925,160,176,33
2,8002,100,612,29.0,16206,16640,303,494,5869,15843,...,17480,16518,268,509,6248,17065,23649,376,338,33
3,8003,100,707,30.0,17784,16759,293,565,6250,17100,...,18126,17173,402,521,6314,16230,25839,604,1497,34
4,8004,100,611,27.0,16341,18377,267,454,5802,13477,...,16993,17113,324,523,6612,11488,20156,406,687,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,100,905,35.0,15440,17688,279,414,6072,13331,...,15060,17845,285,449,6519,11433,21529,743,1835,35
1996,9996,100,560,28.0,19123,18273,291,541,6542,13467,...,17433,18545,328,566,6238,14474,23791,76,1566,36
1997,9997,100,510,25.0,14719,17211,282,512,5914,7891,...,17129,18888,269,507,5629,14181,18033,523,394,35
1998,9998,100,556,30.0,18353,18298,344,625,6043,13838,...,15134,16823,293,495,6042,12048,23098,70,679,34


**The following code calculate the NN classification for league of leguends**

In [12]:
#y_train_timeline = y_train_timeline.sample(n=8000, random_state=345)

In [13]:
X_train = X_teams
y_train = y_train

X_test = y_teams_test

In [14]:
y_train

0       200
1       200
2       200
3       100
4       200
       ... 
7995    200
7996    100
7997    200
7998    200
7999    100
Name: winner, Length: 8000, dtype: int64

In [15]:
pipe = Pipeline(
    steps = [
        ('scaler', MinMaxScaler()),
        ('nn', MLPClassifier(hidden_layer_sizes = (2,),
                             activation = 'tanh',
                             max_iter = 10000))
    ]
)

In [16]:
pipe.fit(X_train, y_train)    

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('nn',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=(2,),
                               max_iter=10000))])

In [17]:
y_pred = pipe.predict(X_test)
y_pred

array([100, 100, 200, ..., 200, 100, 100])

In [18]:
accuracy_score(y_train, pipe.predict(X_train))


0.710875

In [19]:
submission['winner'] = y_pred
submission.head()

,matchId,winner
0,8000,100
1,8001,100
2,8002,200
3,8003,100
4,8004,200


In [20]:
#submission.to_csv('ds6-league-of-legends/1submission.csv', index = False)

# Tried to fit a model with the test data 

The results are much lower. 

In [21]:
variables = ['summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg',
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl'
             ]

# found out later these are the important ones:
# variables = ['summonerLevel', 'championLevel', 'final_gold', 'final_xp', 'final_armor', 'final_health', 'final_atkdmg']


X_test = pd.merge(X_test_original, Y_test_timeline, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_test = X_test.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_test = (
    X_test
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

#X_test = X_test.groupby('matchId')[variables].sum().reset_index(drop = True)

**Testing something different since the predictions are low still**

# Trying a new model to test better results



In [22]:
data = pd.json_normalize(champions['data'])
data

,version,id,key,name,title,blurb,tags,partype,info.attack,info.defense,...,stats.hpregen,stats.hpregenperlevel,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed
0,13.5.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,8,4,...,3.00,1.00,0.00,0.00,0,0,60,5.00,2.500,0.651
1,13.5.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"[Mage, Assassin]",Mana,3,4,...,2.50,0.60,8.00,0.80,0,0,53,3.00,2.000,0.668
2,13.5.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,[Assassin],Energy,5,3,...,9.00,0.90,50.00,0.00,0,0,62,3.30,3.200,0.625
3,13.5.1,Akshan,166,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","[Marksman, Assassin]",Mana,0,0,...,3.75,0.65,8.20,0.70,0,0,52,3.50,4.000,0.638
4,13.5.1,Alistar,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"[Tank, Support]",Mana,6,9,...,8.50,0.85,8.50,0.80,0,0,62,3.75,2.125,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,13.5.1,Zeri,221,Zeri,The Spark of Zaun,"A headstrong, spirited young woman from Zaun's...",[Marksman],Mana,8,5,...,3.25,0.70,6.00,0.80,0,0,53,1.30,2.000,0.658
158,13.5.1,Ziggs,115,Ziggs,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...",[Mage],Mana,2,4,...,6.50,0.60,8.00,0.80,0,0,54,3.10,2.000,0.656
159,13.5.1,Zilean,26,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...","[Support, Mage]",Mana,2,5,...,5.50,0.50,11.35,0.80,0,0,52,3.00,2.130,0.625
160,13.5.1,Zoe,142,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...","[Mage, Support]",Mana,1,7,...,6.50,0.60,8.00,0.65,0,0,58,3.30,2.500,0.625


In [23]:
data


,version,id,key,name,title,blurb,tags,partype,info.attack,info.defense,...,stats.hpregen,stats.hpregenperlevel,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed
0,13.5.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,8,4,...,3.00,1.00,0.00,0.00,0,0,60,5.00,2.500,0.651
1,13.5.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"[Mage, Assassin]",Mana,3,4,...,2.50,0.60,8.00,0.80,0,0,53,3.00,2.000,0.668
2,13.5.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,[Assassin],Energy,5,3,...,9.00,0.90,50.00,0.00,0,0,62,3.30,3.200,0.625
3,13.5.1,Akshan,166,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","[Marksman, Assassin]",Mana,0,0,...,3.75,0.65,8.20,0.70,0,0,52,3.50,4.000,0.638
4,13.5.1,Alistar,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"[Tank, Support]",Mana,6,9,...,8.50,0.85,8.50,0.80,0,0,62,3.75,2.125,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,13.5.1,Zeri,221,Zeri,The Spark of Zaun,"A headstrong, spirited young woman from Zaun's...",[Marksman],Mana,8,5,...,3.25,0.70,6.00,0.80,0,0,53,1.30,2.000,0.658
158,13.5.1,Ziggs,115,Ziggs,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...",[Mage],Mana,2,4,...,6.50,0.60,8.00,0.80,0,0,54,3.10,2.000,0.656
159,13.5.1,Zilean,26,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...","[Support, Mage]",Mana,2,5,...,5.50,0.50,11.35,0.80,0,0,52,3.00,2.130,0.625
160,13.5.1,Zoe,142,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...","[Mage, Support]",Mana,1,7,...,6.50,0.60,8.00,0.65,0,0,58,3.30,2.500,0.625


In [24]:
data['key'] = data['key'].astype(int)

variables = ['participantId','summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg', 'final_atkspd', 
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl',
             'info.attack', 'info.defense', 'stats.hpregen', 'stats.mpregen', 'stats.attackdamage', 'stats.attackspeed']

X_new_train = pd.merge(X_train_original, data, how='outer', left_on='championId', right_on='key')

X_new_train = pd.merge(X_new_train, X_train_timeline, how='inner', on=['matchId', 'participantId'])
X_new_train = pd.merge(X_new_train, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_new_train = X_new_train.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_new_train = (
    X_new_train
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'participantId':'max',
        'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum',
          'stats.attackspeed': 'sum',
          'info.attack': 'sum',
          'stats.mpregen': 'sum',
          'stats.attackdamage': 'sum',
          'info.defense': 'sum',
          'stats.hpregen': 'sum',
          'final_atkspd': 'sum'})
    .reset_index()
)


#convert_team_values(X_train, variables)

#X = X.groupby('matchId')[variables].sum().reset_index(drop = True)

In [25]:
X_test_original

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId
0,8000,100,1,23938,57,Fiora,114
1,8000,100,2,8037,788,LeeSin,64
2,8000,100,3,8444,566,Veigar,45
3,8000,100,4,24831,542,Twitch,29
4,8000,100,5,23890,224,Soraka,16
...,...,...,...,...,...,...,...
19995,9999,200,6,45320,458,Mordekaiser,82
19996,9999,200,7,45321,330,FiddleSticks,9
19997,9999,200,8,45322,462,Tristana,18
19998,9999,200,9,45323,256,Samira,360


In [26]:
team1_new = X_new_train[X_new_train['teamId'] == 100]
team1_new

,matchId,teamId,participantId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,...,final_truedmgdone,final_truedmgtaken,final_lvl,stats.attackspeed,info.attack,stats.mpregen,stats.attackdamage,info.defense,stats.hpregen,final_atkspd
0,0,100,5,860,30.0,14146,17159,316,409,6268,...,375,940,34,3.232,18,34.40,293,23,30.00,691
2,1,100,5,570,33.0,17662,17770,255,511,5636,...,1272,959,34,3.198,33,33.45,292,17,30.50,738
4,2,100,5,619,25.0,16374,16837,292,556,5913,...,195,483,32,3.303,30,33.90,289,19,24.50,658
6,3,100,5,515,28.0,19247,17562,348,531,6240,...,1020,890,34,3.279,30,23.40,304,21,28.50,744
8,4,100,5,747,28.0,16776,18264,304,453,6292,...,163,800,36,3.253,24,41.61,284,19,31.25,630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15990,7995,100,5,772,30.0,19009,18803,290,531,6395,...,94,722,36,3.246,25,40.50,294,24,25.25,728
15992,7996,100,5,533,28.0,16413,17298,426,426,6367,...,309,1015,33,3.350,29,29.91,299,23,33.00,647
15994,7997,100,5,567,29.0,17745,17090,302,556,5918,...,1249,1715,34,3.200,30,34.50,296,27,30.25,634
15996,7998,100,5,689,26.0,15543,15853,277,411,6585,...,211,346,32,3.324,23,78.81,290,23,34.25,595


In [27]:
team2_new = X_new_train[X_new_train['teamId'] == 200]
team2_new

,matchId,teamId,participantId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,...,final_truedmgdone,final_truedmgtaken,final_lvl,stats.attackspeed,info.attack,stats.mpregen,stats.attackdamage,info.defense,stats.hpregen,final_atkspd
1,0,200,10,532,30.0,17539,18773,289,579,6040,...,943,374,35,3.240,34,82.15,305,28,25.75,668
3,1,200,10,567,25.0,14816,18028,316,463,6414,...,959,1272,34,3.262,27,37.60,304,26,29.25,635
5,2,200,10,658,24.0,17347,17467,317,527,6500,...,483,195,34,3.408,30,39.00,312,25,32.75,716
7,3,200,10,599,31.0,15145,16196,301,530,5760,...,890,1020,32,3.189,35,41.00,299,25,29.75,582
9,4,200,10,352,21.0,15385,17194,300,583,5817,...,800,163,33,3.231,38,75.25,294,15,26.75,718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15991,7995,200,10,898,30.0,14873,16977,304,509,5916,...,723,94,34,3.462,29,37.45,310,20,26.50,658
15993,7996,200,10,873,30.0,16140,17927,315,573,6110,...,1015,309,34,3.224,36,64.50,305,27,32.25,748
15995,7997,200,10,464,29.0,16571,15721,352,534,5940,...,1715,1248,33,3.296,36,28.95,300,23,33.50,711
15997,7998,200,10,1165,32.0,18311,18654,278,553,6067,...,347,211,36,3.257,32,35.55,294,24,29.75,624


In [28]:
X_new = pd.merge(team1_new, team2_new, how='outer', on=['matchId'])

X_new

,matchId,teamId_x,participantId_x,summonerLevel_x,championLevel_x,final_gold_x,final_xp_x,final_armor_x,final_atkdmg_x,final_health_x,...,final_truedmgdone_y,final_truedmgtaken_y,final_lvl_y,stats.attackspeed_y,info.attack_y,stats.mpregen_y,stats.attackdamage_y,info.defense_y,stats.hpregen_y,final_atkspd_y
0,0,100,5,860,30.0,14146,17159,316,409,6268,...,943,374,35,3.240,34,82.15,305,28,25.75,668
1,1,100,5,570,33.0,17662,17770,255,511,5636,...,959,1272,34,3.262,27,37.60,304,26,29.25,635
2,2,100,5,619,25.0,16374,16837,292,556,5913,...,483,195,34,3.408,30,39.00,312,25,32.75,716
3,3,100,5,515,28.0,19247,17562,348,531,6240,...,890,1020,32,3.189,35,41.00,299,25,29.75,582
4,4,100,5,747,28.0,16776,18264,304,453,6292,...,800,163,33,3.231,38,75.25,294,15,26.75,718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,100,5,772,30.0,19009,18803,290,531,6395,...,723,94,34,3.462,29,37.45,310,20,26.50,658
7996,7996,100,5,533,28.0,16413,17298,426,426,6367,...,1015,309,34,3.224,36,64.50,305,27,32.25,748
7997,7997,100,5,567,29.0,17745,17090,302,556,5918,...,1715,1248,33,3.296,36,28.95,300,23,33.50,711
7998,7998,100,5,689,26.0,15543,15853,277,411,6585,...,347,211,36,3.257,32,35.55,294,24,29.75,624


In [39]:
data['key'] = data['key'].astype(int)

variables = ['participantId','summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg', 'final_atkspd', 
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl',
             'info.attack', 'info.defense', 'stats.hpregen', 'stats.mpregen', 'stats.attackdamage', 'stats.attackspeed']

X_new_test = pd.merge(X_train_original, data, how='inner', left_on='championId', right_on='key')

X_new_test = pd.merge(X_new_test, X_train_timeline, how='inner', on=['matchId', 'participantId'])
X_new_test = pd.merge(X_new_test, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_new_test =X_new_test.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)

X_new_test = (
   X_new_test
    .groupby(['matchId', 'teamId'])[variables]
    .agg({'participantId':'max',
        'summonerLevel': 'max',
          'championLevel': 'sum',
          'final_gold' : 'sum',
          'final_xp': 'sum',
          'final_armor': 'sum',
          'final_atkdmg': 'sum',
          'final_health': 'sum',
          'final_dmgdone': 'sum',
          'final_dmgtaken': 'sum',
          'final_truedmgdone': 'sum',
          'final_truedmgtaken': 'sum',
          'final_lvl': 'sum',
          'stats.attackspeed': 'sum',
          'info.attack': 'sum',
          'stats.mpregen': 'sum',
          'stats.attackdamage': 'sum',
          'info.defense': 'sum',
          'stats.hpregen': 'sum',
          'final_atkspd': 'sum'})
    .reset_index()
)


In [40]:
X_new_test

,matchId,teamId,participantId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,...,final_truedmgdone,final_truedmgtaken,final_lvl,stats.attackspeed,info.attack,stats.mpregen,stats.attackdamage,info.defense,stats.hpregen,final_atkspd
0,0,100,5,860,30.0,14146,17159,316,409,6268,...,375,940,34,3.232,18,34.40,293,23,30.00,691
1,0,200,10,532,30.0,17539,18773,289,579,6040,...,943,374,35,3.240,34,82.15,305,28,25.75,668
2,1,100,5,570,33.0,17662,17770,255,511,5636,...,1272,959,34,3.198,33,33.45,292,17,30.50,738
3,1,200,10,567,25.0,14816,18028,316,463,6414,...,959,1272,34,3.262,27,37.60,304,26,29.25,635
4,2,100,5,619,25.0,16374,16837,292,556,5913,...,195,483,32,3.303,30,33.90,289,19,24.50,658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,7997,200,10,464,29.0,16571,15721,352,534,5940,...,1715,1248,33,3.296,36,28.95,300,23,33.50,711
15996,7998,100,5,689,26.0,15543,15853,277,411,6585,...,211,346,32,3.324,23,78.81,290,23,34.25,595
15997,7998,200,10,1165,32.0,18311,18654,278,553,6067,...,347,211,36,3.257,32,35.55,294,24,29.75,624
15998,7999,100,5,991,26.0,17058,19319,304,517,5805,...,228,591,35,3.237,27,81.50,308,16,36.50,592


In [41]:
team1_new_test = X_new_test[X_new_test['teamId'] == 100]
team1_new_test

,matchId,teamId,participantId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,...,final_truedmgdone,final_truedmgtaken,final_lvl,stats.attackspeed,info.attack,stats.mpregen,stats.attackdamage,info.defense,stats.hpregen,final_atkspd
0,0,100,5,860,30.0,14146,17159,316,409,6268,...,375,940,34,3.232,18,34.40,293,23,30.00,691
2,1,100,5,570,33.0,17662,17770,255,511,5636,...,1272,959,34,3.198,33,33.45,292,17,30.50,738
4,2,100,5,619,25.0,16374,16837,292,556,5913,...,195,483,32,3.303,30,33.90,289,19,24.50,658
6,3,100,5,515,28.0,19247,17562,348,531,6240,...,1020,890,34,3.279,30,23.40,304,21,28.50,744
8,4,100,5,747,28.0,16776,18264,304,453,6292,...,163,800,36,3.253,24,41.61,284,19,31.25,630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15990,7995,100,5,772,30.0,19009,18803,290,531,6395,...,94,722,36,3.246,25,40.50,294,24,25.25,728
15992,7996,100,5,533,28.0,16413,17298,426,426,6367,...,309,1015,33,3.350,29,29.91,299,23,33.00,647
15994,7997,100,5,567,29.0,17745,17090,302,556,5918,...,1249,1715,34,3.200,30,34.50,296,27,30.25,634
15996,7998,100,5,689,26.0,15543,15853,277,411,6585,...,211,346,32,3.324,23,78.81,290,23,34.25,595


In [42]:
team2_new_test = X_new_test[X_new_test['teamId'] == 200]
team2_new_test

,matchId,teamId,participantId,summonerLevel,championLevel,final_gold,final_xp,final_armor,final_atkdmg,final_health,...,final_truedmgdone,final_truedmgtaken,final_lvl,stats.attackspeed,info.attack,stats.mpregen,stats.attackdamage,info.defense,stats.hpregen,final_atkspd
1,0,200,10,532,30.0,17539,18773,289,579,6040,...,943,374,35,3.240,34,82.15,305,28,25.75,668
3,1,200,10,567,25.0,14816,18028,316,463,6414,...,959,1272,34,3.262,27,37.60,304,26,29.25,635
5,2,200,10,658,24.0,17347,17467,317,527,6500,...,483,195,34,3.408,30,39.00,312,25,32.75,716
7,3,200,10,599,31.0,15145,16196,301,530,5760,...,890,1020,32,3.189,35,41.00,299,25,29.75,582
9,4,200,10,352,21.0,15385,17194,300,583,5817,...,800,163,33,3.231,38,75.25,294,15,26.75,718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15991,7995,200,10,898,30.0,14873,16977,304,509,5916,...,723,94,34,3.462,29,37.45,310,20,26.50,658
15993,7996,200,10,873,30.0,16140,17927,315,573,6110,...,1015,309,34,3.224,36,64.50,305,27,32.25,748
15995,7997,200,10,464,29.0,16571,15721,352,534,5940,...,1715,1248,33,3.296,36,28.95,300,23,33.50,711
15997,7998,200,10,1165,32.0,18311,18654,278,553,6067,...,347,211,36,3.257,32,35.55,294,24,29.75,624


In [43]:
X_new_test = pd.merge(team1_new_test, team2_new_test, how='outer', on=['matchId'])

X_new_test

,matchId,teamId_x,participantId_x,summonerLevel_x,championLevel_x,final_gold_x,final_xp_x,final_armor_x,final_atkdmg_x,final_health_x,...,final_truedmgdone_y,final_truedmgtaken_y,final_lvl_y,stats.attackspeed_y,info.attack_y,stats.mpregen_y,stats.attackdamage_y,info.defense_y,stats.hpregen_y,final_atkspd_y
0,0,100,5,860,30.0,14146,17159,316,409,6268,...,943,374,35,3.240,34,82.15,305,28,25.75,668
1,1,100,5,570,33.0,17662,17770,255,511,5636,...,959,1272,34,3.262,27,37.60,304,26,29.25,635
2,2,100,5,619,25.0,16374,16837,292,556,5913,...,483,195,34,3.408,30,39.00,312,25,32.75,716
3,3,100,5,515,28.0,19247,17562,348,531,6240,...,890,1020,32,3.189,35,41.00,299,25,29.75,582
4,4,100,5,747,28.0,16776,18264,304,453,6292,...,800,163,33,3.231,38,75.25,294,15,26.75,718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,100,5,772,30.0,19009,18803,290,531,6395,...,723,94,34,3.462,29,37.45,310,20,26.50,658
7996,7996,100,5,533,28.0,16413,17298,426,426,6367,...,1015,309,34,3.224,36,64.50,305,27,32.25,748
7997,7997,100,5,567,29.0,17745,17090,302,556,5918,...,1715,1248,33,3.296,36,28.95,300,23,33.50,711
7998,7998,100,5,689,26.0,15543,15853,277,411,6585,...,347,211,36,3.257,32,35.55,294,24,29.75,624


In [44]:
X_new_test.shape

(8000, 43)

In [50]:
x_train = X_new_test
y_train = y_train

x_test = X_new_test

In [51]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report

new_pipe = Pipeline(
    steps = [
        ('impute', SimpleImputer()), #fills average of the column if there are NaN missing values
        ('scaler', StandardScaler()),
        ('log_reg', LogisticRegression(C = 0.0001)) #started from 0.01, 0,001, 
    ]
).fit(X_train,y_train)


In [52]:
scal_pred = new_pipe.predict(X_test)
scal_pred

array([100, 100, 100, ..., 200, 100, 100])

In [53]:
accuracy_score(y_train, scal_pred)

ValueError: Found input variables with inconsistent numbers of samples: [8000, 2000]

In [54]:
print(classification_report(y_train, scal_pred))

ValueError: Found input variables with inconsistent numbers of samples: [8000, 2000]

In [55]:
new_pipe['scaler'].get_feature_names_out()
new_pipe['log_reg'].coef_.flatten()


array([-0.00889097,  0.        ,  0.00027603, -0.03289398, -0.07372453,
       -0.03902779, -0.02162152, -0.02654221, -0.0341113 , -0.05010692,
        0.02711166, -0.01135616,  0.01460931, -0.02887029,  0.        ,
        0.00385716,  0.02834268,  0.07411003,  0.04131565,  0.02090776,
        0.02091645,  0.03765156,  0.0503031 , -0.02385212,  0.01513042,
       -0.01032362,  0.02961342])

In [56]:
pd.DataFrame({'feature': X_train.columns,
                             'coefficient': new_pipe['log_reg'].coef_.flatten()}).sort_values('coefficient', 
                                                                                                      ascending =False).iloc[0:20]

,feature,coefficient
17,final_gold_y,0.074110
22,final_dmgdone_y,0.050303
18,final_xp_y,0.041316
21,final_health_y,0.037652
26,final_lvl_y,0.029613
16,championLevel_y,0.028343
10,final_dmgtaken_x,0.027112
20,final_atkdmg_y,0.020916
19,final_armor_y,0.020908
24,final_truedmgdone_y,0.015130


# trying something new

the following code test a neural network to improve the model predictions. Neural networks optimize their coefficients, it is useful to scale your variable values. In this case, we'll go with a MinMaxScaler, which will convert all variables to values between 0 and 1.

In [57]:
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold

In [58]:
data['key'] = data['key'].astype(int)

variables = ['participantId','summonerLevel', 'championLevel',  'final_gold', 'final_xp', 'final_armor', 'final_atkdmg', 'final_atkspd', 
             'final_health', 'final_dmgdone', 'final_dmgtaken', 'final_truedmgdone', 'final_truedmgtaken', 'final_lvl',
             'info.attack', 'info.defense', 'stats.hpregen', 'stats.mpregen', 'stats.attackdamage', 'stats.attackspeed']

categorical_variables = submission['winner']

In [59]:
pipe = Pipeline(
    steps = [
        ('pf', PolynomialFeatures(interaction_only = True, include_bias = False)),
        ('vt', VarianceThreshold()),
        ('scaler', StandardScaler()),
        ('logistic', LogisticRegression())
    ]
)

pipe.fit(x_train, y_train)

/Users/alisoncordoba/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('pf',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('vt', VarianceThreshold()), ('scaler', StandardScaler()),
                ('logistic', LogisticRegression())])

In [60]:
scal_pred = pipe.predict(X_new_test)
scal_pred

array([200, 100, 200, ..., 100, 200, 100])

In [61]:
accuracy_score(y_train, scal_pred)

0.730125

In [63]:
#submission['winner'] = scal_pred
#submission.head()